<a href="https://colab.research.google.com/github/nedlecky/CSC485B/blob/main/CSC485_121_PythagorasMLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC 485B Spring 2023: CSC485_120_Pythagoras using MLP
## Can we figure out the Pythagorean Theorem with an MLP?
### Input the length of the two sides, ML computes hypotenuse, perimeter, and area
* SUNY Plattsburgh, Spring 2023
* Dr. Ned Lecky
* nleck001@plattsburgh.edu
* ned@lecky.com

In [1]:
# Setup and Support Functions
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# This makes us reproducible (and we can adjust fixed_seed to get different results)
fixed_seed = 1
np.random.seed(fixed_seed)

# Return n random floats between lo and hi as 1-column NumPy matrix
def rand_nlohi(n=1, lo=0, hi=1):
  # This is just a uniform distribution from lo to hi... we can adjust if appropriate in the future
  return (np.random.rand(n) * (hi - lo) + lo).reshape(-1,1)

def nprint(m, name=''):
  print(f"{name} {m.shape} {m.dtype}")
  print(m)



In [2]:
# Test input data for right triangles
# Reminder: We tell you the length of the two sides, you compute length of hypotenuse, perimeter, and area
# x = [side1, side2]
# y = [hypotenuse, perimeter, area]
N = 40
side1 = rand_nlohi(N,1,100)
side2 = rand_nlohi(N,1,100)
X = np.hstack([side1, side2])


In [3]:
# Now let's compute the expected results
# Reminder: We tell you the length of the two sides, you compute length of hypotenuse, perimeter, and area
# x = [side1, side2]
# y = [hypotenuse, perimeter, area]
hypotenuse = np.sqrt(np.square(side1) + np.square(side2))
perimeter = side1 + side2 + hypotenuse
area = (side1 * side2) / 2.
Y = np.hstack([hypotenuse, perimeter, area])


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Let's get it out where we can play in Excel?
triangle_mlp_data = pd.DataFrame(np.hstack([X, Y]), columns=['side1','side2','hyp','perim','area'])
triangle_mlp_data.to_csv('triangle_mlp_data.csv')

triangle_mlp_data.to_csv('drive/MyDrive/triangle_mlp_data.csv')
triangle_mlp_data.to_excel('drive/MyDrive/triangle_mlp_data.xlsx')


In [6]:
# Neural Networks like inputs to be scaled down to +/- 1
from sklearn import preprocessing

std_scaler = preprocessing.StandardScaler()
std_scaler.fit(X)
X_std = std_scaler.transform(X)
#print(np.hstack([X,X_std]))

In [8]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# This is a train/test function that lets us pass in hyperparameters so we can trial
def try_MLP(X, Y, hidden_layer_sizes=(5,2),
            activation='relu', max_iter=200, title='Predictions'):
  global regr # leaving it global so we can use the (last) one in additional tests
  regr = MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=hidden_layer_sizes,
                     activation=activation,
                     max_iter=max_iter,
                     random_state=1)
  regr.fit(X, Y)

  # Add the reshape since decision tree outputs 18,1 otherwise
  Y_pred = regr.predict(X)
  print(f"Mean squared error: {mean_squared_error(Y, Y_pred):.2f}")
  print(f"Mean absolute error: {mean_absolute_error(Y, Y_pred):.2f}")
  print(f"Mean absolute percentage error: {mean_absolute_percentage_error(Y, Y_pred):.2f}")

  # Put error in pandas so we can use .describe()
  #e1 = pd.DataFrame(Y_pred[:0] - Y[:0], columns=['deltaHyp'])
  #print(e1.describe())

# What's ReLU (Rectified Linear Unit)
# https://machinelearningmastery.com/rectified-linear-activation-function-for-deep-learning-neural-networks/

# Hyperparameter experiments
hyperparameter_trials = [
    [(7,4),'relu',200],
    [(8,5),'relu',200],
    [(12,6),'relu',200],
    [(20,20),'relu',200],
    [(40,40),'identity',200],
    [(40,40),'logistic',1000],
    [(40,40),'relu',1000],
    [(40,40),'relu',10000],
]

for hyperparam in hyperparameter_trials:
  print(f"Trying: {hyperparam}")
  try_MLP(X_std, Y, hidden_layer_sizes=hyperparam[0], activation=hyperparam[1], max_iter=hyperparam[2], title=f"Predictions {hyperparam}")


Trying: [(7, 4), 'relu', 200]


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  opt_res = scipy.optimize.minimize(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of i

Mean squared error: 640.29
Mean absolute error: 18.96
Mean absolute percentage error: 0.18
Trying: [(8, 5), 'relu', 200]
Mean squared error: 1544.96
Mean absolute error: 20.75
Mean absolute percentage error: 0.13
Trying: [(12, 6), 'relu', 200]
Mean squared error: 570.92
Mean absolute error: 16.40
Mean absolute percentage error: 0.11
Trying: [(20, 20), 'relu', 200]
Mean squared error: 44.99
Mean absolute error: 4.06
Mean absolute percentage error: 0.02
Trying: [(40, 40), 'identity', 200]
Mean squared error: 43691.90
Mean absolute error: 99.09
Mean absolute percentage error: 0.60
Trying: [(40, 40), 'logistic', 1000]
Mean squared error: 36450.11
Mean absolute error: 97.26
Mean absolute percentage error: 0.67
Trying: [(40, 40), 'relu', 1000]
Mean squared error: 0.64
Mean absolute error: 0.39
Mean absolute percentage error: 0.00
Trying: [(40, 40), 'relu', 10000]
Mean squared error: 0.49
Mean absolute error: 0.35
Mean absolute percentage error: 0.00


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [9]:
# That looks pretty good... let's try some examples not from the dataset
# Here's a 3,4 triangle- expect hypotenuse = 5, perimeter = 3 + 4 + 5 = 12, area = 6
X_345 = np.array([3,4]).reshape(1,-1)
nprint(X_345,'X_345')

# Don't forget to scale the inputs the same way we did for training!
Y_pred = regr.predict(std_scaler.transform(X_345))
nprint(Y_pred,'Y_pred 345')


X_345 (1, 2) int64
[[3 4]]
Y_pred (1, 3) float64
[[ 15.13140054  30.73078325 -12.54381308]]


In [10]:
# Not thrilled... how about for large triangles??
X_304050 = X_345*10;
nprint(X_304050,'X_304050')

# Don't forget to scale the inputs the same way we did for training!
Y_pred = regr.predict(std_scaler.transform(X_304050))
nprint(Y_pred,'Y_pred 304050')


X_304050 (1, 2) int64
[[30 40]]
Y_pred (1, 3) float64
[[ 48.75359696 120.17843703 668.69497035]]


In [22]:
# OK... how about much larger data??
def make_dataset(n = 40):
  global N
  N = n
  # Initially... not so great on small or extra large
  side1 = rand_nlohi(N,1,100)
  side2 = rand_nlohi(N,1,100)
  #side1 = rand_nlohi(N,10,500)
  #side2 = rand_nlohi(N,10,500)
  X = np.hstack([side1, side2])

  hypotenuse = np.sqrt(np.square(side1) + np.square(side2))
  perimeter = side1 + side2 + hypotenuse
  area = (side1 * side2) / 2.
  Y = np.hstack([hypotenuse, perimeter, area])

  return X,Y


# Initially... maybe need even more?
#X, Y = make_dataset(100)
X, Y = make_dataset(300)

std_scaler = preprocessing.StandardScaler()
std_scaler.fit(X)
X_std = std_scaler.transform(X)

# Hyperparameter experiments
hyperparameter_trials = [
    [(20,20),'relu',5000],
    [(20,100,20),'relu',10000],
]

for hyperparam in hyperparameter_trials:
  print(f"Trying: {hyperparam}")
  try_MLP(X_std, Y, hidden_layer_sizes=hyperparam[0], activation=hyperparam[1], max_iter=hyperparam[2], title=f"Predictions {hyperparam}")



Trying: [(20, 20), 'relu', 5000]
Mean squared error: 12.63
Mean absolute error: 1.81
Mean absolute percentage error: 0.01
Trying: [(20, 100, 20), 'relu', 10000]
Mean squared error: 0.31
Mean absolute error: 0.25
Mean absolute percentage error: 0.00


/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:536: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  opt_res = scipy.optimize.minimize(


In [18]:
Y_pred = regr.predict(std_scaler.transform(X_345))
nprint(Y_pred,'Y_pred 345')
Y_pred = regr.predict(std_scaler.transform(X_304050))
nprint(Y_pred,'Y_pred 304050')


Y_pred 345 (1, 3) float64
[[10.96748477 24.87594221 15.33728894]]
Y_pred 304050 (1, 3) float64
[[ 49.92791735 119.8172458  601.71845211]]


In [23]:
# And how is larger still?
X_300400500 = X_345*100;
nprint(X_300400500,'X_300400500')

# Don't forget to scale the inputs the same way we did for training!
Y_pred = regr.predict(std_scaler.transform(X_300400500))
nprint(Y_pred,'Y_pred 300400500')


X_300400500 (1, 2) int64
[[300 400]]
Y_pred 300400500 (1, 3) float64
[[  531.68362302  1221.55871023 26078.21936878]]


In [ ]:
# So would the additional polynomial inputs help??